# Test OneLake Query

Query data from Fabric OneLake using Python + Azure Identity

In [ ]:
# Install required packages (run once)
# !pip install azure-identity pyarrow pandas requests

In [ ]:
import requests
from azure.identity import DefaultAzureCredential
import pyarrow.parquet as pq
import pandas as pd
import io

# Configuration
WORKSPACE_ID = '9c727ce4-5f7e-4008-b31e-f3e3bd8e0adc'
LAKEHOUSE_ID = '06dc42ac-4151-4bb9-94fb-1a03edf49600'

# Get Azure credentials
credential = DefaultAzureCredential()
token = credential.get_token('https://storage.azure.com/.default')
print('Token acquired!')

In [ ]:
# Helper function to read parquet from OneLake
def read_onelake_parquet(file_path: str) -> pd.DataFrame:
    """Read a parquet file from OneLake and return as DataFrame"""
    base_url = f'https://onelake.dfs.fabric.microsoft.com/{WORKSPACE_ID}/{LAKEHOUSE_ID}'
    download_url = f'{base_url}/{file_path}'
    headers = {'Authorization': f'Bearer {token.token}'}
    
    resp = requests.get(download_url, headers=headers)
    if resp.status_code != 200:
        raise Exception(f'Failed to download: {resp.status_code} - {resp.text[:200]}')
    
    buffer = io.BytesIO(resp.content)
    return pq.read_table(buffer).to_pandas()

def list_onelake_files(directory: str, recursive: bool = True) -> list:
    """List files in an OneLake directory"""
    base_url = f'https://onelake.dfs.fabric.microsoft.com/{WORKSPACE_ID}/{LAKEHOUSE_ID}'
    url = f'{base_url}?resource=filesystem&recursive={str(recursive).lower()}&directory={directory}'
    headers = {
        'Authorization': f'Bearer {token.token}',
        'x-ms-version': '2021-06-08'
    }
    
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        raise Exception(f'Failed to list: {resp.status_code}')
    
    return resp.json().get('paths', [])

In [ ]:
# List available data files
files = list_onelake_files('Files/landing/dealer_scrapes', recursive=True)
parquet_files = [f for f in files if f['name'].endswith('.parquet')]
print(f'Found {len(parquet_files)} parquet files')

# Show unique dealers
dealers = set()
for f in parquet_files:
    parts = f['name'].split('/')
    if len(parts) > 4:
        dealers.add(parts[4])  # dealer name is 5th part

print(f'\nDealers ({len(dealers)}): {", ".join(sorted(dealers)[:10])}...')

In [ ]:
# Read sample inventory data - first 10 rows from AC Nelsen (2026-01-01)
file_path = 'Files/landing/dealer_scrapes/ac_nelsen/2026/01/01/acn_20260101.parquet'
df = read_onelake_parquet(file_path)

print(f'Shape: {df.shape}')
print(f'\nColumns: {list(df.columns)}')
print(f'\n=== First 10 Rows ===')
df.head(10)

In [ ]:
# Show key inventory metrics
print('Sample Inventory Summary:')
print(f"- Total units: {len(df)}")
print(f"- Unique makes: {df['make'].nunique()}")
print(f"- Avg sale price: ${df['sale_price'].dropna().astype(float).mean():,.2f}")
print(f"- RV Classes: {df['class'].unique().tolist()}")